In [1]:
import os
import sys

sys.path.insert(0, '../src')

import torch.optim as optim
from torch.utils.data import DataLoader
from model import InpainTor
from losses import CompositeLoss
from dataset import RORDDataset
import optuna

ImportError: cannot import name 'CompositeLoss' from 'losses' (/home/tiagociiic/Projects/InpainTor/notebooks/../src/losses.py)

In [6]:
from graphviz import Digraph

# Criar grafo simplificado
dot = Digraph(comment='InpainTor Simplificado')

# Definir estilos
styles = {
    'input': {'shape': 'box', 'style': 'filled', 'color': 'lightblue2'},
    'output': {'shape': 'box', 'style': 'filled', 'color': 'lightgreen'},
    'default': {'shape': 'ellipse', 'style': 'filled', 'color': 'white'}
}


def apply_styles(node_id, graph, style_dict):
    for key, value in style_dict.items():
        graph.node(node_id, _attributes={key: value})


# Nós
nodes = {
    'A': ('Input Image\n(3, 512, 512)', 'input'),
    'B': ('SharedEncoder\n(enc1, enc2, enc3, enc4)', 'default'),
    'C': ('SegmentorDecoder\n(mask_out, seg2, seg3)', 'default'),
    'D': ('GenerativeDecoder\n(inpainted_image)', 'default'),
    'E': ('Mask Out\n(80, 256, 256)', 'output'),
    'F': ('Inpainted Image\n(3, 256, 256)', 'output'),
    'enc2': ('enc2\n(64, 128, 128)', 'default'),
    'enc3': ('enc3\n(128, 64, 64)', 'default'),
    'enc4': ('enc4\n(256, 32, 32)', 'default'),
    'seg2': ('seg2\n(128, 128, 128)', 'default'),
    'seg3': ('seg3\n(64, 256, 256)', 'default'),
}

# Adicionar nós ao grafo
for node_id, (label, style) in nodes.items():
    dot.node(node_id, label)
    apply_styles(dot.node(node_id), styles.get(style, styles['default']))

# Conexões
edges = [
    ('A', 'B'),
    ('B', 'C'),
    ('B', 'D'),
    ('C', 'E'),
    ('C', 'seg2'),
    ('C', 'seg3'),
    ('C', 'D'),
    ('D', 'F')
]

# Adicionar conexões ao grafo
for start, end in edges:
    dot.edge(start, end)

# Renderizar grafo
dot.render('inpaintor_model_simplified', format='png', view=True)


TypeError: apply_styles() missing 1 required positional argument: 'style_dict'

In [7]:
# Dataset directory configuration
external_disk_path = "/media/tiagociiic/easystore"
split = 'debug'
dataset_dir = os.path.join(external_disk_path, "RORD")
train_dir = os.path.join(dataset_dir, split)
print(f"Dataset directory: {dataset_dir}")

Dataset directory: /media/tiagociiic/easystore/RORD


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the dataset
train_dataset = RORDDataset(root_dir=dataset_dir, split='debug', image_size=[512, 512])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


def train_epoch(model, optimizer, criterion, dataloader):
    model.train()
    total_loss = 0
    for batch in dataloader:
        inputs, seg_gt, inpaint_gt = [batch[k].to(device) for k in ['image', 'mask', 'gt']]
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, seg_gt, inpaint_gt)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


def objective(trial):
    model = InpainTor().to(device)
    criterion = CompositeLoss(seg_loss=torch.nn.NLLLoss(), inpaint_loss=torch.nn.MSELoss(), lambda_=0.5)

    # Suggested learning rate
    lr = trial.suggest_float("lr", 1e-6, 1e-1)

    optimizer = optim.AdamW(model.parameters(), lr=lr)

    # Train the model
    n_epochs = 3
    losses = []
    for epoch in range(n_epochs):
        loss = train_epoch(model, optimizer, criterion, train_loader)
        losses.append(loss)

    return sum(losses) / len(losses)  # Loss averaged over all epochs


# Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Best learning rate and loss
best_lr = study.best_params['lr']
best_loss = study.best_value

print(f"Best learning rate: {best_lr}")
print(f"Best loss: {best_loss}")

final_model = InpainTor().to(device)
final_optimizer = optim.AdamW(final_model.parameters(), lr=best_lr)

2024-07-02 08:32:19.417 | INFO     | dataset:__init__:94 - Loading image, ground truth, and mask files...
2024-07-02 08:32:19.419 | INFO     | dataset:__init__:99 - Loading image files from: /media/tiagociiic/easystore/RORD/debug/img, ground truth files from: /media/tiagociiic/easystore/RORD/debug/gt, and mask files from: /media/tiagociiic/easystore/RORD/debug/multiclass_mask
2024-07-02 08:32:19.422 | INFO     | dataset:__init__:110 - Found 10 image files
2024-07-02 08:32:19.422 | INFO     | dataset:__init__:111 - Found 10 ground truth files
2024-07-02 08:32:19.422 | INFO     | dataset:__init__:112 - Found 10 mask files
2024-07-02 08:32:19.423 | INFO     | dataset:__init__:133 - Found 10 image files, 10 ground truth files, and 10 mask files
2024-07-02 08:32:19.423 | INFO     | dataset:__init__:140 - Found 10 common files in the directories
2024-07-02 08:32:19.425 | INFO     | dataset:__init__:162 - Found 10 valid samples out of 10 total samples
[I 2024-07-02 08:32:19,425] A new study c

Best learning rate: 0.09806447731899928
Best loss: 1.8408315844006007
